In [1]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
import networkx as nx

In [2]:
data_path = '/notebooks/social network/HW1/data/'
p1 = 'Period1.csv'
p2 = 'Period2.csv'
test = 'TestData.csv'

In [3]:
df = pd.read_csv('process1.csv')
df.head()

,source id,target id,common_neigh,jaccard,resource_alloc,adamic_adar,pref_attach,label
0,9802067,9910135,0,0.000000,0.000000,0.000000,1,0
1,9406055,9910067,0,0.000000,0.000000,0.000000,1,0
2,103230,112191,7555,0.999603,0.999821,846.010967,57100690,1
3,9704098,9412062,0,0.000000,0.000000,0.000000,1,0
4,9708149,9412062,0,0.000000,0.000000,0.000000,1,0


In [4]:
X = df.iloc[:,2:-2]
Y = df.iloc[:,-1]

In [5]:
%%time
clf = SVC()
clf.fit(X, Y)

CPU times: user 524 ms, sys: 12 ms, total: 536 ms
Wall time: 533 ms


In [6]:
test_df = pd.read_csv(data_path+test)
test_df.head()

,year,source id,target id
0,2002,209255,8560839
1,2002,205106,6255854
2,2002,212282,2447140
3,2002,209241,4171994
4,2003,301161,9207016


In [7]:
test_df.shape

(10000, 3)

In [8]:
p1_df = pd.read_csv(data_path+p1)
p1_target = set(p1_df['target id'].values.tolist())

In [9]:
p2_df = pd.read_csv(data_path+p2)
p2_target = set(p2_df['target id'].values.tolist())

In [10]:
all_target = list(p1_target|p2_target)

In [11]:
def to_graph(df):
    G=nx.Graph()
    for i in range(df.shape[0]):
        G.add_edge(df.iloc[i]['source id'],df.iloc[i]['target id'])
        
    return G

In [12]:
p1_nonedge = df[df['label']==0]

In [13]:
p1_nonedge.head()

,source id,target id,common_neigh,jaccard,resource_alloc,adamic_adar,pref_attach,label
0,9802067,9910135,0,0.0,0.0,0.0,1,0
1,9406055,9910067,0,0.0,0.0,0.0,1,0
3,9704098,9412062,0,0.0,0.0,0.0,1,0
4,9708149,9412062,0,0.0,0.0,0.0,1,0
5,9909003,9912266,0,0.0,0.0,0.0,1,0


In [14]:
p1_nonedge_G = to_graph(p1_nonedge)

In [15]:
true_p1_edges = list(nx.non_edges(p1_nonedge_G))

In [16]:
len(true_p1_edges)

100

In [17]:
p2_edge = df[df['label']==1]
p2_edge.head()

,source id,target id,common_neigh,jaccard,resource_alloc,adamic_adar,pref_attach,label
2,103230,112191,7555,0.999603,0.999821,846.010967,57100690,1
8,2190,104067,7548,0.998677,0.998894,845.227076,57047793,1
10,9706186,107081,7547,0.998545,0.998762,845.115099,57040226,1
12,2067,3216,7541,0.997751,0.997966,844.443017,56994950,1
14,2073,106276,7549,0.998809,0.999027,845.339075,57055360,1


In [18]:
%%time
p2_edge_G = to_graph(p2_edge)

CPU times: user 2.67 s, sys: 12 ms, total: 2.68 s
Wall time: 2.68 s


In [19]:
len(p2_edge_G.edges())

9998

In [20]:
%%time
new_network_edge = nx.complement(p2_edge_G)
len(new_network_edge.edges())

CPU times: user 1min 26s, sys: 4.18 s, total: 1min 31s
Wall time: 1min 31s


In [21]:
new_network_edge.add_edges_from(true_p1_edges)

In [22]:
len(new_network_edge.edges())

28563100

In [23]:
# def search_in_history(u,v,whicherror):
#     if whicherror=='common_neigh':
#         if v in all_target:
#             return 7558
#         else:
#             return 0
    
#     else if whicherror=='jaccard':
#         if v in all_target:
#             return 1.0
#         else:
#             return 0
        
    

# def compute_common_neigh(G, nonedge):
#     common_neigh = []
#     for e in nonedge:
#         try:
#             cn = len(list(nx.common_neighbors(G,e[0],e[1])))
            
#         except :
#             cn = search_in_history(e[0],e[1],'common_neigh')
#             common_neigh.append((e[0],e[1],cn))
                     
#         else:
# #             cn*=7000
#             common_neigh.append((e[0],e[1],cn))
    
#     return common_neigh



def compute_common_neigh(G, nonedge):
    common_neigh = [(e[0], e[1], len(list(nx.common_neighbors(G,e[0],e[1])))) for e in nonedge]
    return common_neigh


def get_list_value(feature_list):
    v = []
    for i in feature_list:
        v.append(i[2])
        
    to_value = pd.Series(v).values
    
    return to_value


def print_output(out):
    print(len(out))
    print(out[:5])

def get_feature(nonedge, G, df):
    #=========common_neigh===========
    common_neigh = compute_common_neigh(G, nonedge)
    v = get_list_value(common_neigh)
    df['common_neigh'] = v
    print_output(common_neigh)
    
    #=========jaccard_coefficient===========
      
    jaccard = list(nx.jaccard_coefficient(G, nonedge))
    v = get_list_value(jaccard)
    df['jaccard'] = v    
    print_output(jaccard)
    
    resource_alloc = list(nx.resource_allocation_index(G, nonedge))
    v = get_list_value(resource_alloc)
    df['resource_alloc'] = v      
    print_output(resource_alloc)
    
    adamic_adar = list(nx.adamic_adar_index(G, nonedge))
    v = get_list_value(adamic_adar)
    df['adamic_adar'] = v       
    print_output(adamic_adar)
    
    pref_attach = list(nx.preferential_attachment(G, nonedge))
    v = get_list_value(pref_attach)
    df['pref_attach'] = v      
    print_output(pref_attach)

In [24]:
test_df = test_df.drop(columns=['year'])

In [25]:
test_df.head()

,source id,target id
0,209255,8560839
1,205106,6255854
2,212282,2447140
3,209241,4171994
4,301161,9207016


In [26]:
def get_row_data(df):
    edge_list = []
    for index, row in df.iterrows():
        edge_list.append((row['source id'],row['target id']))
        
    return edge_list

In [27]:
edge_list = get_row_data(test_df)

In [28]:
len(list(new_network_edge.nodes()))

7668

In [29]:
node_list = [ int(i)  for i in list(new_network_edge.nodes())]

In [32]:
%%time
label_list = []
cc = 0
for i,v in enumerate(edge_list):
    if (v[0] not in node_list) or (v[1] not in node_list):
        if v[1] in all_target:
            edge_list[i] = (9601024, 6005)
            label_list.append(1)

        else:
            edge_list[i] = (9601024, 9306120)
            label_list.append(0)
    else:
        label_list.append(1)
        cc+=1
        
    if (i+1)%1000==0:    
        print(str(i+1)+' done')

1000 done
2000 done
3000 done
4000 done
5000 done
6000 done
7000 done
8000 done
9000 done
10000 done
CPU times: user 23.6 s, sys: 24 ms, total: 23.6 s
Wall time: 23.6 s


In [33]:
cc

442

In [31]:
# %%time
# label_list = []
# for i,v in enumerate(edge_list):
#     if v[1] in all_target:
#         edge_list[i] = (9601024, 6005)
#         label_list.append(1)

#     else:
#         edge_list[i] = (9601024, 9306120)
#         label_list.append(0)

In [32]:
%%time
get_feature(edge_list, new_network_edge, test_df)

10000
[(9601024, 9306120, 0), (9601024, 9306120, 0), (9601024, 9306120, 0), (9601024, 9306120, 0), (9601024, 6005, 7558)]
10000
[(9601024, 9306120, 0.0), (9601024, 9306120, 0.0), (9601024, 9306120, 0.0), (9601024, 9306120, 0.0), (9601024, 6005, 1.0)]
10000
[(9601024, 9306120, 0), (9601024, 9306120, 0), (9601024, 9306120, 0), (9601024, 9306120, 0), (9601024, 6005, 1.0002171146691166)]
10000
[(9601024, 9306120, 0), (9601024, 9306120, 0), (9601024, 9306120, 0), (9601024, 9306120, 0), (9601024, 6005, 846.346853261051)]
10000
[(9601024, 9306120, 7558), (9601024, 9306120, 7558), (9601024, 9306120, 7558), (9601024, 9306120, 7558), (9601024, 6005, 57123364)]
CPU times: user 14min 12s, sys: 632 ms, total: 14min 13s
Wall time: 14min 13s


In [33]:
test_df

,source id,target id,common_neigh,jaccard,resource_alloc,adamic_adar,pref_attach
0,209255,8560839,0,0.0,0.000000,0.000000,7558
1,205106,6255854,0,0.0,0.000000,0.000000,7558
2,212282,2447140,0,0.0,0.000000,0.000000,7558
3,209241,4171994,0,0.0,0.000000,0.000000,7558
4,301161,9207016,7558,1.0,1.000217,846.346853,57123364
5,202022,9705036,7558,1.0,1.000217,846.346853,57123364
6,212061,4021502,0,0.0,0.000000,0.000000,7558
7,205306,4870136,0,0.0,0.000000,0.000000,7558
8,206243,286390,0,0.0,0.000000,0.000000,7558
9,112112,105316,7558,1.0,1.000217,846.346853,57123364


In [34]:
test_X = test_df.iloc[:,2:-1]

In [35]:
pre = clf.predict(test_X)

In [36]:
pd.Series(pre).value_counts()

0    6407
1    3593
dtype: int64

In [37]:
pd.Series(label_list).value_counts()

0    6407
1    3593
dtype: int64

In [38]:
col = ['target id', 'label']
target = list(range(1,test_df.shape[0]+1))

In [41]:
df = pd.DataFrame({'target id':target,
                   'label':pre
                  },columns=col)

In [42]:
df.to_csv('sub_toogod.csv', index=False)